# Input data (i.e. the data uploaded by Chacoma)

1. `data/`     
    1. `matches.csv` : This table provides metadata for each match. In particular, it specifies the visitor team, the winner team, the score of each team (home and away), the theam names and the league.
  
                match,home,away,winner,score_home,score_away,label,liga
                2500089,1646,1659,1659,1,2,"Burnley - AFC Bournemouth, 1 - 2",England
                2500090,1628,1627,1628,2,0,"Crystal Palace - West Bromwich Albion, 2 - 0",England
                2500091,1673,1609,1609,0,1,"Huddersfield Town - Arsenal, 0 - 1",England
                2500092,1612,1651,1612,4,0,"Liverpool - Brighton & Hove Albion, 4 - 0",England
                ...

    2. `metrics.csv` : This table indicates the metrics obtained by the "team 1" of each match.
  
                match,team1,direct_play,counterattack,T_maintenance,T_build_up,T_zona_media,flow_rate,crossing,pressure_point,pressure_loss,shots,liga
                2499719,1609,1.0464919962132821,0.13796715875566243,0.00801145809451094,1.641360172114294,0.40462924985340015,2.3651718758999496,0.4757393095090685,0.5535438340916512,-0.9998128270169061,2.012366557586264,England
                2499719,1631,-1.6151810636078114,-1.387848393169805,-1.442052913029062,0.599849349015801,-1.3447911625920201,-0.5811499218773863,-0.9819604709287517,-1.2749665117663773,-0.6175176803083413,-2.0563069528271645,England
                2499720,1625,1.7151907876160248,0.17396537163655532,-1.1231815938926129,2.261789632432619,2.2087214645456106,2.0627095788921133,0.992866536319653,1.9518356058153141,1.6004977831784202,0.787573788388354,England
                ...
  
    3. `estilos.csv` : This table indicates the first four "styles" of the "team 1" for each match. A "style" is defined in [].
       
                 ,S1,S2,S3,S4,match,team1,liga
                0,1.1930542934708126,-2.3600679528860655,-0.8587113413310674,-2.1010152816318173,2499719,1609,England
                1,-3.0412107013241307,0.44448036248519524,0.2900507424950798,1.9245273600973778,2499719,1631,England
                2,3.031944843187026,-3.725800852761157,1.432199838655371,-0.29598981396314716,2499720,1625,England
                ...



    4. `league2team2ptos.json` : Dictionary of dictionaries, establishing the mapping "league" -> "team" -> "points" indicating the number of points earned by the corresponding team within the corresponding league. The true-ranks are determined with these points.
  
                {"Spain": {"674": 73, "675": 76, "676": 93, "677": 29, "678": 43, "679": 79, "680": 58, "682": 61, "683": 20, "684": 60, "687": 49, "691": 49, "692": 49, "695": 46, "696": 47, "698": 55, "701": 51, "712": 43, "714": 22, "756": 51}, "England": {"1609": 63, "1610": 70, "1611": 81, "1612": 75, "1613": 44, "1619": 36, "1623": 49, "1624": 77, "1625": 100, "1627": 31, "1628": 44, "1631": 47, "1633": 42, "1639": 33, "1644": 41, "1646": 54, "1651": 40, "1659": 44, "1673": 37, "10531": 33}, "Italy": {"3157": 64,
                ...

    5. `ligue2team2rank.json` : Dictionary of dictionaries, establishing the mapping "league" -> "team" -> "rank" indicating the true-rank of the corresponding team within the corresponding league.
  
                {"Spain": {"676": 1, "679": 2, "675": 3, "674": 4, "682": 5, "684": 6, "680": 7, "698": 8, "701": 9, "756": 10, "687": 11, "691": 12, "692": 13, "696": 14, "695": 15, "678": 16, "712": 17, "677": 18, "714": 19, "683": 20}, "England": {"1625": 1, "1611": 2, "1624": 3, "1612": 4, "1610": 5, "1609": 6, "1646": 7, "1623": 8, "1631": 9, "1613": 10, "1628": 11, "1659": 12, "1633": 13, "1644": 14, "1651": 15, "1673": 16, "1619": 17, "1639": 18, "10531": 19, "1627": 20}, "Italy": {"3159": 1,
  
2. `redes/`

    1. `England/`

        1. `red_counterattack.dat` : Network generated from the metric "counterattack".

                n1,n2,weight
                1,2,2.1776e+00
                1,3,-8.7233e-01
                ...

            Here, `n1` and `n2` are teams and `weight` is the value associated to the metric. There are two entries for each pair of teams, say `weight[n1,n2]` and `weight[n2,n1]`. ...
 
        2. `red_direct_play.dat` : ...
        3. `red_pressure_loss.dat` : ...
        4. `red_shots.dat` : ...
        5. `red_T_maintenance.dat` : ...
        6. `red_crossing.dat` :
        7. `red_flow_rate.dat` :
        8. `red_pressure_point.dat` : ...
        9. `red_T_build_up.dat` : ...
        10. `red_T_zona_media.dat` : ...
    2. `France/` ...
    3. `Germany/` ...
    4. `Italy/` ...
    5. `Spain/`...
  
               
## Output data

The input data is invariably composed of values $m_{ij}$, where $i$ and $j$ represent two teams and $m_{ij}$ represents the weight associated to the pair of matches between them (one where $i$ is the home team and $j$ is the away team and another where it is the other way around) for a given metric $m$, say "counterattack".
Generally, the data consider all $N(N-1)/2$ pairings for the the $N=n+1$ teams in a league.
In other words, the $N(N-1)/2$ values $m_{ij}$ for $i<j$ form the lower triangle of a matrix.

For each metric $m$ of each league, the output data consists of an undirected weighted graph $\mathcal{G}$ of links $ij$ with $i<j$ and weights
$$
f_{ij} = -\frac{1}{2}(m_{ij}+m_{ji}).
$$
Eventually, the 1-chain $f=f_{ij}e^{ij}\in C_1^*$ is created from the weighted graph $\mathcal{G}$.

Which 1-cochains are created?
1. the metric-based defined in:
    1. `redes/England/red_shots.dat`, etc.
    2. `data/estilos.csv`.
2. the ground truth:
    1. point based coefficients $f_{ij} = \pi_j-\pi_i$,
    2. rank based coefficients $f_{ij}=i-j$,

### Sparsification

In some cases, we may consider the generation of sparsified versions of $f_{ij}$ where not all $N(N-1)/2$ pairs $ij$ for $i<j$ are provided.

### Output format

In all cases, the output format chosen for $f$ is a weighted non-directed graph where the $ij$ link for $i<j$ has weight $f_{ij}$.

In [ ]:
# Run this cell in a folder contained by the current project
using Pkg
#Pkg.activate()                 # Walks up the path tree until finds Project.toml
#Pkg.activate(".")              # 
#Pkg.activate("@__DIR__")       # 
Pkg.activate("../")            #
Pkg.instantiate()

In [ ]:
using Plots
using LaTeXStrings
using Graphs
#using MetaGraphs
#using MetaGraphsNext
using GraphPlot
using SimpleWeightedGraphs
using Karnak
using NetworkLayout
using Colors
using Random
using LinearAlgebra
using OffsetArrays
using SparseArrays
using Serialization
using IterativeSolvers
using Krylov
using BenchmarkTools
using Profile
using ProfileView
using Statistics
using DifferentialEquations
using LoopVectorization
using Folds
using Transducers
using Distributed
using JLD2
using OnlineStats
using ImageFiltering
using SavitzkyGolay
using Interpolations
using DataInterpolations
using Polynomials
using CurveFit
using DataFrames
using LsqFit
using ForwardDiff
using JSON

In [ ]:
function enum!(d,k)
    return get!(d,k,length(d)+1)
end

### Test

In [ ]:
mutable struct RunningStat
    n::Int
    m::Real # mean
    s::Real # std
end

function RunningStat()
    return RunningStat(0,0.0,0.0)
end

function Base.push!(rs::RunningStat,v::Real)
    if rs.n>0
        rs.n += 1
        new_m = rs.m + (v-rs.m)/rs.n
        rs.s += (v-rs.m)*(v-new_m)
        rs.m = new_m
    else
        rs.n = 1
        rs.m = v
        rs.s = 0
    end
    return rs.m
end

function Statistics.mean(rs::RunningStat)
    return rs.m 
end

function Statistics.var(rs::RunningStat)
    return rs.s/(rs.n-1)
end

function Statistics.std(rs::RunningStat)
    return sqrt(var(rs))
end

### Tests

In [ ]:
list_v = 10 .+ 5.0*randn(1000)
;

In [ ]:
rs = RunningStat()

In [ ]:
for v in list_v
    push!(rs,v)
end

In [ ]:
mean(rs),std(rs)

In [ ]:
mean(list_v),std(list_v)

In [ ]:
function R_squared(y_data,y_fit)
    y_mean = mean(y_data)
    SS_res = sum((y_data .- y_fit) .^ 2)
    SS_tot = sum((y_data .- y_mean) .^ 2)
    R_sqr = 1 - SS_res / SS_tot
    return R_sqr
end

### Tests

In [ ]:
# Define the model function
@. model(x, p) = p[1] * x + p[2]

# Sample data (example)
x_data = [1.0, 2.0, 3.0, 4.0, 5.0]
y_data = [2.2, 2.8, 3.6, 4.5, 5.1]

# Initial guess for parameters
p0 = [1.0, 1.0]

# Perform curve fitting
fit = LsqFit.curve_fit(model, x_data, y_data, p0)

# Compute fitted values
y_fit = model(x_data, coef(fit))

# Compute R_sqr
R_sqr = R_squared(y_data,y_fit)

In [ ]:
plot()
#plot!(yscale=:log10)
#plot!(ylim=(0.0,1.0))
plot!(xlabel=L"x")
plot!(ylabel=L"y")
scatter!(x_data,y_data,label="data")
plot!(x_data,y_fit,label="fit, \$R^2=0.995\$")
#plot!(0.0:0.0,x->0.0,label="\$z=1\$",width=0.0,c=:white)
#plot!(legendtitle=L"\mathrm{1D},\;z=1",legendtitlefontsize=8)
#plot!(list_sigma,x->0.6827/2,label="",style=:dash,color=:grey)
#plot!(
#    xticks=([0,1,2,3,4,5],[L"0",L"1",L"2",L"3",L"4",L"5"]),
#    yticks=([0,1,2,3],[L"0",L"1",L"2",L"3"]),
#    tickfont=font(11),
#)

In [ ]:
x0 = 1.0
A = 1.0
B = 3.0
C = 0.0
p0 = [x0,A,B,C] # p1=x0, p2=A, p3=B, p4=C
@. sigmoid(x,p)  = p[2]/(1+exp(-p[3]*(x-p[1]))) # sigmoid
@. dsigmoid(x,p) = p[2]*p[3]*exp(-p[3]*(x-p[1]))/(1+exp(-p[3]*(x-p[1])))^2 # derivative of sigmoid
@. psigmoid(x,p) = p[2]/p[3]*log(1+exp(p[3]*(x-p[1])))+p[4] # primitive of sigmoid

### Test

In [ ]:
plot(
    ylims=(-0.1,1.1),
    xlabel=L"x",
    ylabel=L"f(x)",
)
plot!(-2:0.01:4.0,x->sigmoid(x,p0),label="")

In [ ]:
plot(
    ylims=(-0.1,1.1),
    xlabel=L"x",
    ylabel=L"df/dx",    
)
plot!(-2:0.01:4.0,x->dsigmoid(x,p0),label="")
scatter!([x0],[A*B/4],label="")

In [ ]:
plot(
    ylims=(-0.1,1.1),
    xlabel=L"x",
    ylabel=L"F(x)",
)
plot!(-2:0.01:4.0,x->psigmoid(x,p0),label="")

# Original format -> Julia's JLD2 format

## `redes/[league]/[metric].dat` to JLD2 format

In [ ]:
list_nets_dat = """../redes/Italy/red_crossing.dat
../redes/Italy/red_counterattack.dat
../redes/Italy/red_pressure_loss.dat
../redes/Italy/red_T_build_up.dat
../redes/Italy/red_direct_play.dat
../redes/Italy/red_pressure_point.dat
../redes/Italy/red_shots.dat
../redes/Italy/red_flow_rate.dat
../redes/Italy/red_T_maintenance.dat
../redes/Italy/red_T_zona_media.dat
../redes/Germany/red_crossing.dat
../redes/Germany/red_counterattack.dat
../redes/Germany/red_pressure_loss.dat
../redes/Germany/red_T_build_up.dat
../redes/Germany/red_direct_play.dat
../redes/Germany/red_pressure_point.dat
../redes/Germany/red_shots.dat
../redes/Germany/red_flow_rate.dat
../redes/Germany/red_T_maintenance.dat
../redes/Germany/red_T_zona_media.dat
../redes/Spain/red_crossing.dat
../redes/Spain/red_counterattack.dat
../redes/Spain/red_pressure_loss.dat
../redes/Spain/red_T_build_up.dat
../redes/Spain/red_direct_play.dat
../redes/Spain/red_pressure_point.dat
../redes/Spain/red_shots.dat
../redes/Spain/red_flow_rate.dat
../redes/Spain/red_T_maintenance.dat
../redes/Spain/red_T_zona_media.dat
../redes/England/red_crossing.dat
../redes/England/red_counterattack.dat
../redes/England/red_pressure_loss.dat
../redes/England/red_T_build_up.dat
../redes/England/red_direct_play.dat
../redes/England/red_pressure_point.dat
../redes/England/red_shots.dat
../redes/England/red_flow_rate.dat
../redes/England/red_T_maintenance.dat
../redes/England/red_T_zona_media.dat
../redes/France/red_crossing.dat
../redes/France/red_counterattack.dat
../redes/France/red_pressure_loss.dat
../redes/France/red_T_build_up.dat
../redes/France/red_direct_play.dat
../redes/France/red_pressure_point.dat
../redes/France/red_shots.dat
../redes/France/red_flow_rate.dat
../redes/France/red_T_maintenance.dat
../redes/France/red_T_zona_media.dat"""

### DEPRECATED 2025-09-01

### NEW VERSION 2025-09-01

In [ ]:
dict_f_ij_metrics_jld2 = Dict()
for archivo in split(list_nets_dat)
    println(archivo)
    vec_i = Vector{Int64}()
    vec_j = Vector{Int64}()
    vec_f_ij = Vector{Float64}()    
    open(archivo) do fh
        for line in readlines(fh)[2:end]
            cols=split(replace(line,"," => " "))
            #println(cols)
            i=parse(Int64,cols[1])
            j=parse(Int64,cols[2])
            #@assert i<j
            mij_plus_mji=parse(Float64,cols[3])
            f_ij = 0.5*mij_plus_mji
            @show i,j,f_ij            
            push!(vec_i,i)
            push!(vec_j,j)
            push!(vec_f_ij,f_ij)
        end
    end
    n = max(maximum(vec_i),maximum(vec_i))
    g = SimpleWeightedGraph(n)
    for (i,j,f_ij) in zip(vec_i,vec_j,vec_f_ij)
        add_edge!(g,i,j,f_ij)
    end
    tag = replace(archivo, "../redes/" => "")
    tag = replace(tag, ".dat" => "")
    tag = replace(tag, "red_" => "")
    tag = replace(tag, "/" => " ")
    tag = split(tag)
    tag = (tag[1],tag[2])
    dict_f_ij_metrics_jld2[tag] = g
end

### Tests

In [ ]:
@save "jld2/dict_f_ij_metrics.jld2" dict_f_ij_metrics_jld2

In [ ]:
@load "jld2/dict_f_ij_metrics.jld2" dict_f_ij_metrics_jld2

### Tests

## DEPRECATED 2025-09-01 ## Load "puntos" per team per league and  save them in JLD2 format

## Load "redes de estilos" and save them in JLD2 format

In [ ]:
files_redes_estilos = split("""../redes_estilos/Italy/red_S2.dat
../redes_estilos/Italy/red_S4.dat
../redes_estilos/Italy/red_S3.dat
../redes_estilos/Italy/red_S1.dat
../redes_estilos/Germany/red_S2.dat
../redes_estilos/Germany/red_S4.dat
../redes_estilos/Germany/red_S3.dat
../redes_estilos/Germany/red_S1.dat
../redes_estilos/Spain/red_S2.dat
../redes_estilos/Spain/red_S4.dat
../redes_estilos/Spain/red_S3.dat
../redes_estilos/Spain/red_S1.dat
../redes_estilos/England/red_S2.dat
../redes_estilos/England/red_S4.dat
../redes_estilos/England/red_S3.dat
../redes_estilos/England/red_S1.dat
../redes_estilos/France/red_S2.dat
../redes_estilos/France/red_S4.dat
../redes_estilos/France/red_S3.dat
../redes_estilos/France/red_S1.dat""")

### DEPRECATED 2025-09-01

### NEW VERSION 2025-09-01

In [ ]:
dict_phi_ij_estilos_jld2 = Dict()
for archivo in files_redes_estilos
    println(archivo)
    vec_i = Vector{Int64}()
    vec_j = Vector{Int64}()
    vec_phi_ij = Vector{Float64}()    
    open(archivo) do fh
        for line in readlines(fh)[2:end]
            cols=split(replace(line,"," => " "))
            #println(cols)
            i = parse(Int64,cols[1])
            j = parse(Int64,cols[2])
            phi_ij = parse(Float64,cols[3])
            #@show i,j,phi_ij
            push!(vec_i,i)
            push!(vec_j,j)
            push!(vec_phi_ij,phi_ij)
        end
    end
    n = max(maximum(vec_i),maximum(vec_i))
    g = SimpleWeightedGraph(n)
    for (i,j,phi_ij) in zip(vec_i,vec_j,vec_phi_ij)
        add_edge!(g,i,j,phi_ij)
    end    
    tag = replace(archivo, "../redes_estilos/" => "")
    tag = replace(tag, ".dat" => "")
    tag = replace(tag, "red_" => "")
    tag = replace(tag, "/" => " ")
    tag = split(tag)
    #@show tag
    tag = (tag[1],tag[2])
    dict_phi_ij_estilos_jld2[tag] = g
end

### Tests

### Save as JLD2

In [ ]:
@save "jld2/dict_phi_ij_estilos.jld2" dict_phi_ij_estilos_jld2

In [ ]:
@load "jld2/dict_phi_ij_estilos.jld2" dict_phi_ij_estilos_jld2

### Tests

## Create "points based" 1-chains and save them in JLD2 format

# References

1. [A. Chacoma and O. V. Billoni, Data-driven approach to defining football styles in major leagues, Chaos, Solitons & Fractals 200, 116926 (2025)](https://doi.org/10.1016/j.chaos.2025.116926)
2. [J. I. Perotti, Analysis of the inference of ratings and rankings in complex networks using discrete exterior calculus on higher-order networks, Phys. Rev. E 111, 034306 (2025)](https://journals.aps.org/pre/abstract/10.1103/PhysRevE.111.034306)